# Imports

In [1]:
# general use libraries
import numpy as np

# Brevitas ad PyTorch libraries
import torch
from torch.nn import Module, Sequential, BatchNorm2d
from brevitas.nn import QuantIdentity, QuantConv2d, QuantLinear, QuantReLU, QuantMaxPool2d

In [3]:
class QTinyYOLOv2(Module):

    def __init__(self, weight_bit_width=8, act_bit_width=8, quant_tensor=True):
        super(QLeNet, self).__init__()
        self.weight_bit_width = int(np.clip(weight_bit_width, 1, 8))
        self.act_bit_width = int(np.clip(act_bit_width, 1, 8))

        self.input = QuantIdentity(
            act_quant=Int8ActPerTensorFloatMinMaxInit,
            min_val = -1.0,
            max_val = 1.0 - 2.0 ** (-7),
            signed = True,
            restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
            return_quant_tensor=quant_tensor
        )
        self.conv1 = Sequential(
            QuantConv2d(3, 16, 3, 1, (2,2), bias=False, weight_bit_width=8, return_quant_tensor=quant_tensor),
            BatchNorm2d(16),
            QuantReLU(bit_width=8, return_quant_tensor=quant_tensor),
            QuantMaxPool2d(2, 2, (1,1), return_quant_tensor=quant_tensor)
        )
        self.conv2 = Sequential(
            QuantConv2d(16, 32, 3, 1, (2,1), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(32),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor),
            QuantMaxPool2d(2, 2, (0,1), return_quant_tensor=quant_tensor)
        )
        self.conv3 = Sequential(
            QuantConv2d(32, 64, 3, 1, (1,1), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(64),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor),
            QuantMaxPool2d(2, 2, (0,1), return_quant_tensor=quant_tensor)
        )
        self.conv4 = Sequential(
            QuantConv2d(64, 128, 3, 1, (2,2), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(128),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor),
            QuantMaxPool2d(2, 2, (0,0), return_quant_tensor=quant_tensor)
        )
        self.conv5 = Sequential(
            QuantConv2d(128, 256, 3, 1, (1,2), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(256),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor),
            QuantMaxPool2d(2, 2, (0,0), return_quant_tensor=quant_tensor)
        )
        self.conv6 = Sequential(
            QuantConv2d(256, 512, 3, 1, (2,2), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(512),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor),
            QuantMaxPool2d(2, 2, (0,0), return_quant_tensor=quant_tensor)
        )
        self.conv7 = Sequential(
            QuantConv2d(512, 512, 3, 1, (2,2), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(512),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor)
        )
        self.conv8 = Sequential(
            QuantConv2d(512, 512, 3, 1, (1,2), bias=False, weight_bit_width=self.weight_bit_width, return_quant_tensor=quant_tensor),
            BatchNorm2d(512),
            QuantReLU(bit_width=self.act_bit_width, return_quant_tensor=quant_tensor)
        )
        self.conv9 = QuantConv2d(512, 5, 3, 1, 0, bias=False, weight_bit_width=8, return_quant_tensor=quant_tensor)
        )

    def forward(self, x):
        x = self.input(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        return x

In [ ]:
def coor2label(xmax, xmin, ymax, ymin, width=640, height=360, w_px=40, h_px=40):
    """ Take bounding box information and return YOLO style label
    
    input:
    xmax   -- high value of bounding box horizontal axis
    xmin   -- low value of bounding box horizontal axis 
    ymax   -- high value of bounding box vertical axis
    ymin   -- low value of bounding box vertical axis 
    width  -- width of image (default value: 640)
    height -- height of image (default value: 360)
    w_px   -- width of grid boxes in pixels (default value: 40 pixels)
    h_px   -- height of grid boxes in pixels (default value: 40 pixels)
    
    output:
    idx -- grid indices of bounding box centre (tuple: (idx_x, idx_y))
    b_x -- bounding box centre x coordinates inside the grid box
    b_y -- bounding box centre y coordinates inside the grid box
    b_w -- bounding box width
    b_h -- bounding box hight
    """
    if xmax>width or xmin>width or ymax>height or ymin>height:
        raise ValueError('At least one of the bounding box sizes exceeds the image shape.')
    elif xmax<0 or xmin<0 or ymax<0 or ymin<0:
        raise ValueError('Bounding box sizes coordinates must be positive.')
    b_w = abs(xmax - xmin)
    b_h = abs(ymax - ymin)
    x_centre = (xmax + xmin) / 2
    y_centre = (ymax + ymin) / 2
    b_x = (x_centre % w_px) / w_px
    b_y = (y_centre % h_px) / h_px
    idx = (np.floor(x_centre / w_px), np.floor(y_centre / h_px))
    return (idx, [b_x, b_y, b_w, b_h])